## Weather Condition & Covid-19 Data analysis :
####   to find the relationship between Environment and the Virus spread

This Notebook is to test if there is a relationship between the Environment(Temperature, Wind, and Humidity) and the spread of Covid_19 through the data. I found that the environment has a negative effect on the spread of the virus meaning as the temperature, wind speed and the humidity increase the spread of the virus decrease, of course, the decrease on the number of new cases reported could be a result of another factor like lock-down measure or so on, but because we cannot afford to conduct a fully controlled test between the environment and the virus spread I use the available data to do the test. Below you find the Data and the test method I used to do the research.
### The Data
##### New Cases Reported per-Day
I calculated the new cases per day for four different States around the world (New York.US,California.US, New South Wales.AS, and Hong Kong).
Data Source: Kaggle.
##### Weather (Tempriture, Wind and Humidity)
I gathered the weather data and calculated the average of the past ten days meaning the value of 22-01 is the average of the past ten days(13 to 22), that's because it's likely the person had the infection during the last ten days.
Data Sources: https://www.ncdc.noaa.gov/, https://www.wunderground.com, and https://www.hko.gov.hk

### The Method
I used the Spearman Coefficient Correlation test, I used it through the Panada implementation Function pd.df.corr(method=' spearman').

In [1]:
# first import the lipraries
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Import the Data:

In [2]:
# Import the data
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
covid_19 = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')

/kaggle/input/weather-data/covid_data/nswa.csv
/kaggle/input/weather-data/covid_data/cal1.csv
/kaggle/input/weather-data/covid_data/weather.csv
/kaggle/input/weather-data/covid_data/all_weather.xlsx
/kaggle/input/weather-data/covid_data/cal2.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv
/kaggle/input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv
/kaggle/input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths_US.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed_US.csv


### The Data

In [3]:
covid_19.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


# New Cases per Day
####  I calculated the number of new cases recorded for each day by calculating the difference between current confirmed cases from the previous day (Confirmed.diff()).
1-Hong Kong.

In [4]:
#Data Preprocessing
grouped = (covid_19.groupby(['Province/State','ObservationDate']).Confirmed.sum()).reset_index()

#_____________Hong Kong New Cases Data______

hong_kong = grouped.loc[grouped['Province/State']=='Hong Kong']
hong_kong.insert(3, 'New_cases', hong_kong[['Confirmed']].diff().fillna(hong_kong[['Confirmed']]), True)
hong_kong = hong_kong.iloc[1:117].reset_index()

hong_kong.head()

,index,Province/State,ObservationDate,Confirmed,New_cases
0,7967,Hong Kong,01/23/2020,2.0,2.0
1,7968,Hong Kong,01/24/2020,2.0,0.0
2,7969,Hong Kong,01/25/2020,5.0,3.0
3,7970,Hong Kong,01/26/2020,8.0,3.0
4,7971,Hong Kong,01/27/2020,8.0,0.0


2-New South Wales, Australia

In [5]:
#_____________NSW New Cases Data_____________

nsw = grouped.loc[grouped['Province/State']=='New South Wales']
nsw.insert(3, 'New_cases', nsw[['Confirmed']].diff().fillna(nsw[['Confirmed']]), True)
nsw = nsw.iloc[0:108].reset_index()
nsw.head()

,index,Province/State,ObservationDate,Confirmed,New_cases
0,14152,New South Wales,01/27/2020,4.0,4.0
1,14153,New South Wales,01/28/2020,4.0,0.0
2,14154,New South Wales,01/29/2020,4.0,0.0
3,14155,New South Wales,01/30/2020,4.0,0.0
4,14156,New South Wales,01/31/2020,4.0,0.0


3-California, US

In [6]:
#____________Newyork State New Cases Data__________

california = grouped.loc[grouped['Province/State']=='California'] 
california.insert(3,'New_cases', california[['Confirmed']].diff().fillna(california[['Confirmed']]), True)
new_cases_ca = california.iloc[6:75,:].reset_index()
new_cases_ca.head()

,index,Province/State,ObservationDate,Confirmed,New_cases
0,2800,California,03/10/2020,144.0,142.0
1,2801,California,03/11/2020,177.0,33.0
2,2802,California,03/12/2020,221.0,44.0
3,2803,California,03/13/2020,282.0,61.0
4,2804,California,03/14/2020,340.0,58.0


4-New York, US

In [7]:
newyork = grouped.loc[grouped['Province/State']=='New York'] 
newyork.insert(3,'New_cases', newyork[['Confirmed']].diff().fillna(newyork[['Confirmed']]), True)
new_cases_ny = newyork.iloc[0:69,:].reset_index()
new_cases_ny.head()

,index,Province/State,ObservationDate,Confirmed,New_cases
0,14293,New York,03/10/2020,173.0,173.0
1,14294,New York,03/11/2020,220.0,47.0
2,14295,New York,03/12/2020,328.0,108.0
3,14296,New York,03/13/2020,421.0,93.0
4,14297,New York,03/14/2020,525.0,104.0


### New Cases Visualization:
States of New York & California has way more new cases reported per day (1000th) compared to the number of new cases reported in New South Wales (100th) & Hong Kong (10th), 


In [8]:
fig = go.Figure()
fig.add_trace(go.Bar(x=hong_kong['ObservationDate'],
                y=hong_kong['New_cases'],
                name='Hong Kong',
                marker_color='blue'
                ))
fig.add_trace(go.Bar(x=nsw['ObservationDate'],
                y=nsw['New_cases'],
                name='New South Wales',
                marker_color='Red'
                ))
fig.update_layout(
    title='Corona Virus New Cases Per Day (Hong Kong & NSW)',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig1 = go.Figure()
fig1.add_trace(go.Bar(x=new_cases_ca['ObservationDate'],
                y=new_cases_ca['New_cases'],
                name='California',
                marker_color='Green'
                ))
fig1.add_trace(go.Bar(x=new_cases_ny['ObservationDate'],
                y=new_cases_ny['New_cases'],
                name='New York',
                marker_color='Black'
#                      'rgb(55, 83, 109)'
                ))
fig1.update_layout(
    title='Corona Virus New Cases Per Day (New York & California)',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()
fig1.show()

# Weather Data:
#### Here I used the average of the past ten days, for example, the Temperature on 25-03 is the average of days between 16-03 and 25-03, the same applies to Humidity and Wind Speed.(Weather.rolling(10).mean())
##### 1-Hong Kong Weather Data:
##### TAVG: Temperature Average.
##### AWND: Wind Speed Avergage.
##### Humidity: Humidity Average.

In [9]:
#____________Hong Kong Weather Data_______________

hk_weather = pd.read_excel('/kaggle/input/weather-data/covid_data/all_weather.xlsx')
hk_weather = hk_weather.rename(columns={'Temp-Mean':'TAVG','Mean Relative Humidity (%)':'Humidity',
                                       'Mean Wind Speed (km/h)':'AWND','Day':'DATE'})
hk_weather = hk_weather.iloc[13:138,:]

hk_weather.loc[:,'TAVG'] = hk_weather['TAVG'].rolling(10).mean()
hk_weather.loc[:,'Humidity'] = hk_weather['Humidity'].rolling(10).mean()
hk_weather.loc[:,'AWND'] = hk_weather['AWND'].rolling(10).mean()

hk_weather = hk_weather.iloc[9::,:].reset_index()

# # #__________Hong Kong Final Data___________

rHongKong = pd.concat([hk_weather[['DATE']],hong_kong[['New_cases']],hk_weather[['TAVG']],
                      hk_weather[['AWND']], hk_weather[['Humidity']]], axis = 1)
rHongKong.head()

,DATE,New_cases,TAVG,AWND,Humidity
0,2020-01-23,2.0,19.25,25.29,78.0
1,2020-01-24,0.0,19.50,25.14,79.3
2,2020-01-25,3.0,19.52,25.09,80.2
3,2020-01-26,3.0,19.19,24.84,80.4
4,2020-01-27,0.0,18.64,25.67,80.5


Hong Kong Weather Data Visualization:

In [10]:

fig = make_subplots(rows=2, cols=1)#, column_widths=[0.7, 0.3])

fig.add_trace(go.Bar(x=rHongKong['DATE'],
                y=rHongKong['AWND'],name='Wind Speed (km/H)',marker_color='green'),
                row=1, col=1)
fig.add_trace(go.Scatter(x=rHongKong['DATE'],
                y=rHongKong['Humidity'],name='Humidity (%)',marker_color='blue'),
                row=2, col=1)
fig.add_trace(go.Bar(x=rHongKong['DATE'],
                y=rHongKong['TAVG'],name='Temperature in (°C)',
                marker=dict(color=rHongKong['TAVG'], coloraxis="coloraxis")),
                row=2, col=1)
              

fig.update_layout(title='Hong Kong Weather')

fig.update_layout(coloraxis=dict(colorscale='Inferno'),legend=dict(x=1,
        y=1.2,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'))
fig.show()

##### 2-New South Wales Weather Data:
##### TAVG: Temperature Average.
##### AWND: Wind Speed Avergage.
##### Humidity: Humidity Average.

In [11]:
#____________NSW Weather Data_______________

nsw_weather = pd.read_csv('/kaggle/input/weather-data/covid_data/nswa.csv')
nswhumid = pd.read_excel('/kaggle/input/weather-data/covid_data/all_weather.xlsx', sheet_name='NSW')

nsw_temp = nsw_weather.groupby(['DATE']).TAVG.mean().reset_index()
nsw_temp.loc[:,'TAVG'] = nsw_temp['TAVG'].rolling(10).mean()
nsw_w = nsw_temp.iloc[9::,:].reset_index()
#Wind
nswwh = nswhumid.set_index(nswhumid['Date'])
nswind = nswwh['Wind'].rolling(10).mean()
rolling_nswind = nswind.iloc[9:117].reset_index()
#Humidity
nswhumid = nswhumid.set_index(nswhumid['Date'])
nswhumidh = nswwh['Humidity'].rolling(10).mean()
rolling_nswhumid = nswhumidh.iloc[9:117].reset_index()

# #NSW Final Data
rnsw = pd.concat([nsw_w[['DATE']],nsw[['New_cases']],nsw_w[['TAVG']],
                 rolling_nswind[['Wind']],rolling_nswhumid[['Humidity']]], axis = 1)
rnsw = rnsw.rename(columns={'Wind':'AWND'})
rnsw.head()

,DATE,New_cases,TAVG,AWND,Humidity
0,2020-01-27,4.0,25.450000,19.360,75.000
1,2020-01-28,0.0,25.938333,19.480,73.850
2,2020-01-29,0.0,26.250000,19.845,73.130
3,2020-01-30,0.0,26.760000,19.330,73.140
4,2020-01-31,0.0,27.388333,20.020,73.495


New South Wales Weather Data Visualization:

In [12]:

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Bar(x=rnsw['DATE'],
                y=rnsw['AWND'],name='Wind Speed (km/H)',marker_color='green'),
                row=1, col=1)
fig.add_trace(go.Scatter(x=rnsw['DATE'],
                y=rnsw['Humidity'],name='Humidity (%)',marker_color='blue'),
                row=2, col=1)
fig.add_trace(go.Bar(x=rnsw['DATE'],
                y=rnsw['TAVG'],name='Temperature (°C)',
                marker=dict(color=rnsw['TAVG'], coloraxis="coloraxis")),
                row=2, col=1)
              

fig.update_layout(title='New South Wales Weather')

fig.update_layout(coloraxis=dict(colorscale='Inferno'),legend=dict(x=1,
        y=1.2,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'))
fig.show()

##### 3-California Weather Data: 
##### TAVG: Temperature Average.
##### AWND: Wind Speed Avergage.
##### Humidity: Humidity Average.

In [13]:
#____________Newyork State New Weather Data__________
#Data Preprocessing
ca_weather_1 = pd.read_csv('/kaggle/input/weather-data/covid_data/cal1.csv')
ca_weather_2 = pd.read_csv('/kaggle/input/weather-data/covid_data/cal2.csv')
cal_weather_n = ca_weather_1[ca_weather_1['DATE']>='2020-03-01']

ca_weather = pd.concat([cal_weather_n,ca_weather_2], axis=0)
cahumid = pd.read_excel('/kaggle/input/weather-data/covid_data/all_weather.xlsx', sheet_name='CA')

#convert mile to km for wind speed
# ca_weather[['AWND']] = ca_weather[['AWND']].apply(lambda w: w*1.609)

ca_temp = ca_weather.groupby('DATE').TAVG.mean()
ca_rolling_temp = ca_temp.rolling(10).mean()
ca_wind = ca_weather.groupby('DATE').AWND.mean()
ca_rolling_wind = ca_wind.rolling(10).mean()
ca_rolling_temp = ca_rolling_temp.iloc[9:78].reset_index()
ca_rolling_wind = ca_rolling_wind.iloc[9:78].reset_index()
#Humidity
cahumid = cahumid.set_index(cahumid['Date'])
chumid = cahumid['Humidity'].rolling(10).mean()
rolling_cahumid = chumid.iloc[9:78].reset_index()
# # California State Final Data 

rcalifornia = pd.concat([ca_rolling_temp[['DATE']],new_cases_ca[['New_cases']],ca_rolling_temp[['TAVG']],ca_rolling_wind[['AWND']],
                        rolling_cahumid[['Humidity']]], axis = 1)
rcalifornia.head()

,DATE,New_cases,TAVG,AWND,Humidity
0,2020-03-10,142.0,8.490856,2.904012,58.273333
1,2020-03-11,33.0,9.100377,2.678001,59.413333
2,2020-03-12,44.0,9.464740,2.529734,63.093333
3,2020-03-13,61.0,9.200531,2.630088,66.723333
4,2020-03-14,58.0,8.497239,2.798678,69.000000


California State Weather Data Visualization:

In [14]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Bar(x=rcalifornia['DATE'],
                y=rcalifornia['AWND'],name='Wind Speed (km/H)',marker_color='green'),
                row=1, col=1)
fig.add_trace(go.Scatter(x=rcalifornia['DATE'],
                y=rcalifornia['Humidity'],name='Humidity (%)',marker_color='blue'),
                row=2, col=1)
fig.add_trace(go.Bar(x=rcalifornia['DATE'],
                y=rcalifornia['TAVG'],name='Temperature (°C)',
                marker=dict(color=rcalifornia['TAVG'], coloraxis="coloraxis")),
                row=2, col=1)
              

fig.update_layout(title='California State Weather')

fig.update_layout(coloraxis=dict(colorscale='Inferno'),legend=dict(x=1,
        y=1.2,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'))
fig.show()

##### 4-New York Weather Data:
##### TAVG: Temperature Average.
##### AWND: Wind Speed Avergage.
##### Humidity: Humidity Average.

In [15]:
#____________Newyork State Weather Data____________

nweather = pd.read_csv('/kaggle/input/weather-data/covid_data/weather.csv')
nyhumid = pd.read_excel('/kaggle/input/weather-data/covid_data/all_weather.xlsx', sheet_name='NY')
#Temp & Wind

#convert mile to km for wind speed
# nweather[['AWND']] = nweather[['AWND']].apply(lambda w: w*1.609)

temp = nweather.groupby('DATE').TAVG.mean()
rolling_temp = temp.rolling(10).mean()
wind = nweather.groupby('DATE').AWND.mean()
rolling_wind = wind.rolling(10).mean()
rolling_temp = rolling_temp.iloc[9:78].reset_index()
rolling_wind = rolling_wind.iloc[9:78].reset_index()
#Humidity
nyhumid = nyhumid.set_index(nyhumid['Date'])
humid = nyhumid['Humidity'].rolling(10).mean()
rolling_humid = humid.iloc[9:78].reset_index()

# #_______Newyork State Final Data 
rnewyork = pd.concat([rolling_temp[['DATE']],new_cases_ny[['New_cases']],rolling_temp[['TAVG']],rolling_wind[['AWND']],
                     rolling_humid[['Humidity']]], axis = 1)
rnewyork.head()

,DATE,New_cases,TAVG,AWND,Humidity
0,2020-03-10,173.0,4.894286,4.121563,55.736667
1,2020-03-11,47.0,5.683571,4.072490,56.400000
2,2020-03-12,108.0,5.714286,3.960302,57.226667
3,2020-03-13,93.0,5.767857,4.307177,56.533333
4,2020-03-14,104.0,5.679286,4.186552,55.570000


New York Weather Data Visualization: 

In [16]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Bar(x=rnewyork['DATE'],
                y=rnewyork['AWND'],name='Wind Speed (km/H)',marker_color='green'),
                row=1, col=1)
fig.add_trace(go.Scatter(x=rnewyork['DATE'],
                y=rnewyork['Humidity'],name='Humidity (%)',marker_color='blue'),
                row=2, col=1)
fig.add_trace(go.Bar(x=rnewyork['DATE'],
                y=rnewyork['TAVG'],name='Temperature (°C)',
                marker=dict(color=rcalifornia['TAVG'], coloraxis="coloraxis")),
                row=2, col=1)
              

fig.update_layout(title='New York State Weather')

fig.update_layout(coloraxis=dict(colorscale='Inferno'),legend=dict(x=1,
        y=1.2,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'))
fig.show()

# Correlation:
To find the relationship between the number of new cases reported and the weather condition variables(Temperature, Humidity & Wind Speed), I used the Spearman correlation test because I'm not sure if the relationship is fully linear between the variables.
The result shows a negative relationship between new cases and Temperature, Humidity & Wind Speed.

##### TAVG: Temperature Average.
##### AWND: Wind Speed Avergage.
##### Humidity: Humidity Average.

In [17]:
#Concatenate all The Data 
#Date
all_dates = pd.concat([rHongKong[['DATE']],
                      rnsw[['DATE']],
                      rcalifornia[['DATE']],
                      rnewyork[['DATE']],
                     ], axis = 0)
#New Cases
all_new_cases = pd.concat([rHongKong[['New_cases']],
                      rnsw[['New_cases']],
                      rcalifornia[['New_cases']],
                      rnewyork[['New_cases']],
                     ], axis = 0)
#Tempriture
all_temp = pd.concat([rHongKong[['TAVG']],rnsw[['TAVG']],
                      rcalifornia[['TAVG']],rnewyork[['TAVG']]
                     ], axis = 0)
#Wind
all_wind = pd.concat([rHongKong[['AWND']],rnsw[['AWND']],
                      rcalifornia[['AWND']],rnewyork[['AWND']]
                     ], axis = 0)
#Humidity
all_humid = pd.concat([rHongKong[['Humidity']],rnsw[['Humidity']],
                      rcalifornia[['Humidity']],rnewyork[['Humidity']]
                     ], axis = 0)
all_data = pd.concat([all_dates,all_new_cases, all_temp, all_wind, all_humid], axis = 1)

## Correlation Test
all_data.corr(method='spearman').style.background_gradient('viridis')

,New_cases,TAVG,AWND,Humidity
New_cases,1.000000,-0.757516,-0.669798,-0.608739
TAVG,-0.757516,1.000000,0.657464,0.721499
AWND,-0.669798,0.657464,1.000000,0.676502
Humidity,-0.608739,0.721499,0.676502,1.000000
